In [30]:
from selenium import webdriver
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pickle
import os.path
from datetime import datetime
import plotly.express as px

In [7]:
#do only for first login! After, comment this part out
driver = webdriver.Chrome()
instagram = "https://www.instagram.com/"
driver.get(instagram)

Now login manually on your instagram account. Normally after the first login, the cookies should be saved in Chrome's folder which are easy to load afterwards. However if this does not work you can the load_pickled_cookies function below for an alternate way to load them.

In [ ]:
#Let's save the cookies. Running these functions is optional, only if loading profiled browser doesn't work
def pickle_cookies(driver):                                          #pickled cookies? mmm must be yummy!
    pickle.dump(driver.get_cookies(), open('cookies.pkl', 'wb'))
def load_pickled_cookies(driver):
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)

In [12]:
def load_profiled_browser():
    options=webdriver.ChromeOptions()
    options.add_argument("user-data-dir=/home/nicolae/.config/google-chrome")
    driver = webdriver.Chrome(options=options)
    return driver 

instagram = "https://www.instagram.com/"
driver = load_profiled_browser()
driver.implicitly_wait(2)
driver.get(instagram)

For the next step, make sure your Chrome window is maximized! If you actually did not post any story and the window isn't maximized, it can click and go through the stories of other people you're following.  

In [17]:
stories_remaining = True
try:
    own_story = driver.find_element_by_xpath('/html/body/div[1]/section/main/section/div[3]/div[1]/div/div[1]/span/img')
    own_story.click()
except NoSuchElementException: 
    try:       #nested, I know, but following EAFP principle
        own_story = driver.find_element_by_xpath('/html/body/div[1]/section/main/section/div[1]/div/div/div/div[1]/button/div[1]/span/img') 
        print("story of others")
        own_story.click()
    except NoSuchElementException:
        stories_remaining = False 
        print("no such element")
viewers = []        
next_story_button = 1
while stories_remaining:     
    try:
        view_count = driver.find_element_by_xpath('/html/body/div[1]/section/div/div/section/div[2]/div[3]/div[2]/button/div/span').text
        view_count = int(view_count.split("Vista por")[1])
        viewers.append(view_count)
        next_story = driver.find_element_by_xpath(f'/html/body/div[1]/section/div/div/section/div[2]/button[{next_story_button}]/div')
        next_story_button = 2
        driver.implicitly_wait(1)
        next_story.click()
    except (NoSuchElementException, StaleElementReferenceException) as e:
        stories_remaining = False
viewers

no such element


[]

In [28]:
driver.close()


WebDriverException: Message: invalid session id
  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Linux 4.15.0-76-generic x86_64)


In [70]:
story_filename = "story_info.csv"
if not os.path.isfile(story_filename):
    story_info = pd.DataFrame(columns=['Date', 'Num. stories', 'Views for each story', "Category", "Completion rate"])
    story_info.to_csv(story_filename, index=False)
else:
    story_info = pd.read_csv(story_filename)
story_info

,Date,Num. stories,Views for each story,Category,Completion rate
0,2020-02-19,8,"[90, 90, 88, 70, 65, 65, 65, 64]",memes,71.111111


In [71]:
def add_to_dataframe(story_info, category=""):
    viewers = [90, 90, 88, 70, 65, 65, 65, 64]           #just sample data, usually this part is scraped with the code above and this line should be commented out
    story_dict = {}
    story_dict["Date"] = datetime.now().date()
    story_dict["Num. stories"] = len(viewers)
    story_dict["Views for each story"] = viewers
    story_dict["Completion rate"] = (viewers[-1]*100) / viewers[0]
    story_dict["Category"] = category
    story_info = story_info.append(story_dict, ignore_index=True)
    return story_info

story_info = add_to_dataframe(story_info, "memes")
story_info

,Date,Num. stories,Views for each story,Category,Completion rate
0,2020-02-19,8,"[90, 90, 88, 70, 65, 65, 65, 64]",memes,71.111111
1,2020-02-19,8,"[90, 90, 88, 70, 65, 65, 65, 64]",memes,71.111111


In [72]:
story_info.to_csv(story_filename, index=False)

In [59]:
story_2 = story_info['Views for each story'][2]
fig.update_layout(title_text='Viewer retention for your story')
fig = px.bar(story_2, y=story_2, labels={'y':'Story viewers', 'x':'Story number'})
fig.show()